In [26]:
# Import libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import nltk
import psycopg2
import nltk
import os

In [79]:
# Importing titles, links and wikipedia synopses
os.chdir("/home/pjoshi/github/data-science/projects/document-clustering")
titles = open('title_list.txt').read().split('\n')
titles = titles[:100]

links = open('link_list_imdb.txt').read().split('\n')
links = links[:100]

synopses_wiki = open('synopses_list_wiki.txt').read().split('\n BREAKS HERE')
synopses_wiki = synopses_wiki[:100]

#strip html code and convert to unicode
synopses_clean_wiki = []
for text in synopses_wiki:
    text = BeautifulSoup(text,'html.parser').getText()
    synopses_clean_wiki.append(text)
    
synopses_wiki = synopses_clean_wiki

genres = open('genres_list.txt').read().split('\n')
genres = genres[:100]

print(str(len(titles)) + ' titles')
print(str(len(links)) + ' links')
print(str(len(synopses_wiki)) + ' synopses')
print(str(len(genres)) + ' genres')


100 titles
100 links
100 synopses
100 genres


In [66]:
os.getcwd()

'/home/pjoshi/github/data-science/projects/document-clustering'

In [80]:
# Import imdb synopses
synopses_imdb = open('synopses_list_imdb.txt').read().split('\n BREAKS HERE')
synopses_imdb = synopses_imdb[:100]

synopses_clean_imdb = []

for text in synopses_imdb:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    synopses_clean_imdb.append(text)

synopses_imdb = synopses_clean_imdb


In [82]:
synopses = []

for i in range(len(synopses_wiki)):
    item = synopses_wiki[i] + synopses_imdb[i]
    synopses.append(item)

In [83]:
# generating index for each item in corpora
ranks = []

for i in range(0,len(titles)):
    ranks.append(i)
    

#### Stemming and Tokenization of Synopses



In [87]:
stopwords = nltk.corpus.stopwords.words('english')


In [89]:
#### Import Snowball stemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [98]:
## Define functions: tokenize_and_stem(text) and tokenize_only(text)

def tokenize_and_stem(text):
    
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    
    for token in tokens:
        if re.search('[a-zA-Z]',token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens
        


In [101]:
#### Iterate over synopses to create 2 vocabularies, one stemed and one only tokenized

totalvocab_stemmed = []
totalvocab_tokenized = []

for i in synopses:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [102]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)


In [ ]:
vocab_frame.head(30)

#### TF-IDF and Document Similarity